# Global indicator project - Phoenix, Arizona

## accessibility analysis - amenities pois
This notebook uses [pandana](https://udst.github.io/pandana/network.html) to download points-of-interest data from OpenStreetMap within a defined study region bounding box.


In [2]:
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import numpy as np
import requests
import pandas as pd
import geopandas as gpd
import os
import time 

import config

import pandana
from pandana.loaders import osm

ox.config(use_cache=True, log_console=True)
pandana.__path__

['/Users/NGAU/miniconda3/envs/ind_global/lib/python3.6/site-packages/pandana']

## Set up configuration 

In [4]:
place = 'phoenix' 

region = 'Arizona, USA' # study region name

studyregion = 'Phoenix, Arizona, USA'

suffix = '_201905'

# configure points-of-interests(pois) for daily living scores
shop = ['supermarket', 'convenience']

# configure filenames to save/load POI and network datasets
OSM_folder = '../data/OSM'

#cofigure pois filename and file path
poi_filename = '{}_pois_{}.csv'.format(place, '_'.join(shop))

poi_filepath = OSM_folder + "/" + poi_filename

In [6]:
# get bounding box from study region boundary shapefile
#  check the shapefile format, study region boundary shapefile should specify lat lon geometry, 
# otherwise, we can use can generate a bounding box by going to http://boundingbox.klokantech.com/ and selecting the CSV format.   
shape_filepath = '../data/OSM/Phoenix, Arizona, USA_buffered_201905/Phoenix, Arizona, USA_buffered_201905.shp'
gdf_shape = gpd.GeoDataFrame.from_file(shape_filepath)

bbox = [gdf_shape['bbox_south'].astype(float)[0], gdf_shape['bbox_west'].astype(float)[0], gdf_shape['bbox_north'].astype(float)[0], gdf_shape['bbox_east'].astype(float)[0]] #lat-long bounding box for Phx
bbox

[33.2903739, -112.3240289, 33.9183794, -111.9255201]

## Download points of interest (POIs) from OSM
What amenities are considered for daily living pois? - [OSMtag](https://taginfo.openstreetmap.org/keys/amenity): shop=supermarket, convenience

In [29]:
def get_osm_pois(poi_filepath=poi_filepath, shop=shop, bbox=bbox):
    if os.path.isfile(poi_filepath):
        # if a points-of-interest file already exists, just load the dataset from that
        pois = pd.read_csv(poi_filepath)
        method = 'loaded from CSV'
    else:   
        # otherwise, query the OSM API for the specified amenities within the bounding box 
        osm_tags = '"shop"~"{}"'.format('|'.join(shop))
        pois = osm.node_query(bbox[0], bbox[1], bbox[2], bbox[3], tags=osm_tags)

        # drop any that aren't just 'shop' then save to CSV
        pois = pois[pois['shop'].isin(shop)]
        pois.to_csv(poi_filepath, index=False, encoding='utf-8')
        method = 'downloaded from OSM'
    pois_df = pois[['shop', 'name', 'lat', 'lon']]
    return pois_df
    

In [30]:
pois_df = get_osm_pois(poi_filepath=poi_filepath, shop=shop, bbox=bbox)
pois_df.head()

403 POIs downloaded from OSM in 0.99 seconds


,shop,name,lat,lon
id,,,,
158170307,supermarket,Safeway,33.488551,-112.081794
194872171,convenience,7-Eleven,33.480013,-112.029747
315987032,supermarket,Fry's Marketplace,33.321899,-111.930303
316912138,supermarket,Fry's Food & Drug,33.379098,-111.976857
319992591,convenience,NaN,33.348719,-111.953919
